<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Background" data-toc-modified-id="Background-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Background</a></span></li><li><span><a href="#Setup" data-toc-modified-id="Setup-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Setup</a></span></li><li><span><a href="#Data" data-toc-modified-id="Data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Data</a></span><ul class="toc-item"><li><span><a href="#About-the-data|" data-toc-modified-id="About-the-data|-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>About the data|</a></span></li><li><span><a href="#Read-data" data-toc-modified-id="Read-data-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Read data</a></span></li><li><span><a href="#Matrix-form" data-toc-modified-id="Matrix-form-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Matrix form</a></span></li></ul></li><li><span><a href="#Aux-functions" data-toc-modified-id="Aux-functions-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Aux functions</a></span></li><li><span><a href="#User-recommendations" data-toc-modified-id="User-recommendations-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>User recommendations</a></span><ul class="toc-item"><li><span><a href="#Product-similarity-based-recommendations" data-toc-modified-id="Product-similarity-based-recommendations-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Product similarity based recommendations</a></span><ul class="toc-item"><li><span><a href="#Distances" data-toc-modified-id="Distances-5.1.1"><span class="toc-item-num">5.1.1&nbsp;&nbsp;</span>Distances</a></span></li><li><span><a href="#Rules-for-similarity-based-recommendations" data-toc-modified-id="Rules-for-similarity-based-recommendations-5.1.2"><span class="toc-item-num">5.1.2&nbsp;&nbsp;</span>Rules for similarity based recommendations</a></span></li><li><span><a href="#Example-1" data-toc-modified-id="Example-1-5.1.3"><span class="toc-item-num">5.1.3&nbsp;&nbsp;</span>Example 1</a></span></li></ul></li><li><span><a href="#User-similarity-based-recommendations" data-toc-modified-id="User-similarity-based-recommendations-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>User similarity based recommendations</a></span></li></ul></li><li><span><a href="#Collaborative-Filtering" data-toc-modified-id="Collaborative-Filtering-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Collaborative Filtering</a></span><ul class="toc-item"><li><span><a href="#Parameters" data-toc-modified-id="Parameters-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Parameters</a></span></li><li><span><a href="#Simple-Gradient-Descent" data-toc-modified-id="Simple-Gradient-Descent-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Simple Gradient Descent</a></span><ul class="toc-item"><li><span><a href="#Initialize" data-toc-modified-id="Initialize-6.2.1"><span class="toc-item-num">6.2.1&nbsp;&nbsp;</span>Initialize</a></span></li><li><span><a href="#Gradient-Descent" data-toc-modified-id="Gradient-Descent-6.2.2"><span class="toc-item-num">6.2.2&nbsp;&nbsp;</span>Gradient Descent</a></span></li></ul></li><li><span><a href="#Regularisation" data-toc-modified-id="Regularisation-6.3"><span class="toc-item-num">6.3&nbsp;&nbsp;</span>Regularisation</a></span></li><li><span><a href="#Stochastic-GD" data-toc-modified-id="Stochastic-GD-6.4"><span class="toc-item-num">6.4&nbsp;&nbsp;</span>Stochastic GD</a></span></li></ul></li></ul></div>

# Background
This is based on ideas from this [post](http://www.albertauyeung.com/post/python-matrix-factorization/), but takes a more simple, script-like approach

# Setup

Data & Algebra libs

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import pairwise_distances

Graphic libs

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy.cluster import hierarchy

# Data

## About the data|
You can find everything you need to know about the MovieLens dataset [here](http://files.grouplens.org/datasets/movielens/ml-100k/README)

## Read data

In [ ]:
ml = \
pd.read_csv(
    filepath_or_buffer = 'http://files.grouplens.org/datasets/movielens/ml-100k/u.data',
    header = None,
    sep = '\t'
)

In [ ]:
ml.columns = 'user id | item id | rating | timestamp'.split(' | ')

For offline usage

In [ ]:
# ml.to_feather('movie_rec.feather')
# ml = pd.read_feather('movie_rec.feather')

Our dataset at a glance

In [ ]:
ml.shape

In [ ]:
ml.head()

In [ ]:
ml.describe()

## Matrix form

In [ ]:
data_mat = \
ml.drop(columns=['timestamp']).\
    pivot(
        index = 'user id', 
        columns = 'item id', 
        values = 'rating'
    ).\
    fillna(0)

# Aux functions

In [ ]:
def vis_dist_mat(dist, bins = 30):
    '''Visualize different aspects of a distance matrix
    
    :param dist: a distance matrix calculated by `pairwise_distances`
    
    :Example:
    >>> vis_dist_mat(dist)
    '''
    
    dist_vec = \
    pd.Series(
        dist[
            np.tril_indices(dist.shape[0], k=1)
        ]
    )
    
    print('Distance Matrix Details\n=======================\n')
    
    print(
        '* have have',
        (dist_vec == 0).sum(),
        'identical products (with distance = 0), and',
        (dist_vec == 1).sum(),
        'with no common ratings (distance = 1)\n'
    )
    print('* Percentiles:')
    print(dist_vec.describe(), '\n')
    
    sns.distplot(
        dist_vec,
        bins = bins,
        kde = False,
        norm_hist = False,
    ).\
    set_title('Distance distribution')
    
    plt.figure()

    dn = hierarchy.dendrogram(hierarchy.ward(dist))
    plt.xticks([])
    plt.title('Cluster dendogram')

    plt.show()
    
    return None

# User recommendations

## Product similarity based recommendations
This approach looks at all the products recommended by the users and try to find "similar" products based on some simple rules.

### Distances

In [ ]:
product_dist = pairwise_distances(X = data_mat.T, metric = 'cosine')

In [ ]:
vis_dist_mat(product_dist)

### Rules for similarity based recommendations

There are several strategies for picking the products we want to recommend based on product similarity, but essentially we need to make several choices
1. What products we look at: all the products the user rated? Only producs with $n$ stars or more? Or do we weights the distances by the number of stars? And even with weights - do 1 star reviews have a positive weight or shoud we give a low number of stars a negative weight? 
2. How do we aggregate the distances? Say the user only rated 2 products, and both of them are similar to product $j$. Do we rank product $j$ compared to all the other producuts that have some similarity to the 2 rated products? Do we sum up distances for each product and rank? Do we take $min$/$max$/... distance for each product?
3. What would be the threshold for a product to be recommended? This is dependent on practical limitaions, e.g. what's the minimum/maximim number of products we have to/can recommend, and what is the distribution of distances and other considerations (e.g. do we want an absolute threshold of similarity and not just the top 5)

Looking at the second choice we have to make: there's one huge advantage to summing up distances over picking min/max or other aggregation methods - __we can use matrix multiplication!__

### Example 1
We decide to look at all the movies a user rated (no matter what the rating was) and sum up the similarities to other products, and recommend the top 5 most similar products (regardless of the absolute level of similarity - in this example we _have_ to recommend exactly 5 products, no matter what)

We define 
1. The symmetric matrix $P$ containing prodcut to product distances as we calculated above (our `product_dist` matrix based on `cosine` distance)
2. A matrix with binary values $C$

\begin{equation*}
    C_{I \times J} = \left( \left( c_{i,j} \right) \right)_{I \times J}
\end{equation*}
where
\begin{equation*}
    c_{i,j} = 
    \left[
        \begin{array}{ll}
          1 & R_{i,j} > 0 \\
          0 & \text{otherwise}
        \end{array}
    \right.
\end{equation*}

For every user $i$ we want to look at all rated products, and find the distances from other products. This can be written as a sparse matrix, where only rows matching rated products will have non-zero values:

\begin{equation*}
   \forall i,  \left( \left( c_{i,j} p_{j,k} \right) \right)_{J \times J}
\end{equation*}

And if our way to rank products is to sum up distances, we can sum up across columns, to get $\left( \sum_{j} C_{i,j} P_{j,1} , \ldots, \sum_{j}  C_{i,j} P_{j,J} \right)$, which is simply the $i$'th row of the matrix $CP$, which we can calculate as:

In [ ]:
CP = (data_mat > 0).dot(product_dist)
CP.columns = data_mat.columns

For practical reasons we need to work with the long format, sorted by similarity.

In [ ]:
user_rec = pd.DataFrame({'dist': CP.T.unstack()})

And now we can pull out only the top 5 most similar products (or apply and other selection rule)

In [ ]:
user_rec.\
groupby(user_rec.index.get_level_values(0)).\
apply(
    lambda x: x.sort_values(by = 'dist').head(5).reset_index()
).head(15)

## User similarity based recommendations

In [ ]:
user_dist = pairwise_distances(X = data_mat, metric = 'cosine')

In [ ]:
vis_dist_mat(user_dist)

For rach user we pick the top $n$ most similar users and select the products they liked most 

# Collaborative Filtering

## Parameters

In [ ]:
K = 3
R = np.array(data_mat)
iterations = 1000
tolerance = 0.05
alpha = 0.0001

## Simple Gradient Descent

### Initialize

In [ ]:
P = np.random.normal(scale=1./K, size=(R.shape[0], K))
Q = np.random.normal(scale=1./K, size=(K, R.shape[1]))

In [ ]:
training_process = pd.DataFrame(columns=['iteration', 'mse'])
t = 1
mse_prev = 0
mse = 1e6

### Gradient Descent

In [ ]:
while t <= iterations and abs(mse - mse_prev) > tolerance:
    mse_prev = mse
    
    ## Error matrix
    error = R - P.dot(Q)

    ## Update down the gradient
    P = P + 2 * alpha * error.dot(Q.T)
    Q = Q + 2 * alpha * P.T.dot(error)

    mse = np.power(error, 2).sum()
    training_process = training_process.append(pd.DataFrame({'iteration': [t], 'mse': [mse]}))

    if (t+1) % 50 == 0:
        print("Iteration: %d ; error = %.4f" % (t+1, mse))
    t += 1

In [ ]:
training_process.set_index('iteration', inplace=True)

In [ ]:
plt.figure()
plt.plot(training_process['mse'])
plt.title('Algorithm progress')

## Regularisation

TBD
* Add a penalty on $\sqrt{\sum q_{i,j}^2 + \sum p_{i,j}^2}$ ($L2$) or $\sum \left| q_{i,j} \right| + \sum \left| p_{i,j} \right| $ ($L1$) with a penalty coeff $\gamma$ on the MSE goal function and optimize $MSE - \lambda ()$ 
* This probably requires some form of normalisation of $P,Q$ prior to optimisation 

## Stochastic GD

TBD